In [1]:
import datetime
import math
import os
import struct

import pandas as pd
import requests

GlobalSession = requests.session()

from constant import TODAY
from quant_utils.constant import DB_CONN_LOCAL_MYSQL
from quant_utils.utils import change_1min_to_other


def get_table_list():

    query_sql = """
    SELECT
        TABLE_NAME 
    FROM
        `information_schema`.`TABLES` 
    WHERE
        1=1
        and `TABLE_SCHEMA` = '1min'
        and TABLE_NAME like "S%"
    """
    return DB_CONN_LOCAL_MYSQL.exec_query(query_sql)["TABLE_NAME"].tolist()

In [2]:
table_list = get_table_list()

In [13]:
table_list.index("SZ399001")

1189

In [14]:
for code in table_list[1189:]:
    ticker = code[2:] + "." + code[0:2]
    print(ticker)
    query = f"""
        select * from {code}
    """
    df = DB_CONN_LOCAL_MYSQL.exec_query(query).set_index("trade_time")
    type_dict = {
        "D": "day",
        "30min": "30min", 
        "60min": "60min",
    }
    for period in ["D", "30min", "60min"]:
        df_other = change_1min_to_other(df, period).reset_index()
        df_other["code"] = ticker
        if period != "D":
            df_other["date"] = df_other["trade_time"].apply(lambda s: s.strftime("%Y%m%d"))
            df_other["tick"] = df_other["trade_time"].apply(lambda s: s.strftime("%H%M%S"))
        DB_CONN_LOCAL_MYSQL.upsert(df_other, table=f"{type_dict[period]}_line")

399001.SZ
(1264, "Out of range value for column 'total_amount' at row 1188")
写入192.20.57.188:1min--30min_line-受影响行数为37200行!
写入192.20.57.188:1min--60min_line-受影响行数为18600行!
399002.SZ
(1264, "Out of range value for column 'total_amount' at row 1188")
写入192.20.57.188:1min--30min_line-受影响行数为37200行!
写入192.20.57.188:1min--60min_line-受影响行数为18600行!
399003.SZ
写入192.20.57.188:1min--day_line-受影响行数为4650行!
写入192.20.57.188:1min--30min_line-受影响行数为37200行!
写入192.20.57.188:1min--60min_line-受影响行数为18600行!
399004.SZ
写入192.20.57.188:1min--day_line-受影响行数为4646行!
写入192.20.57.188:1min--30min_line-受影响行数为37168行!
写入192.20.57.188:1min--60min_line-受影响行数为18584行!
399005.SZ
写入192.20.57.188:1min--day_line-受影响行数为3802行!
写入192.20.57.188:1min--30min_line-受影响行数为30416行!
写入192.20.57.188:1min--60min_line-受影响行数为15208行!
399006.SZ
写入192.20.57.188:1min--day_line-受影响行数为3336行!
写入192.20.57.188:1min--30min_line-受影响行数为26688行!
写入192.20.57.188:1min--60min_line-受影响行数为13344行!
399007.SZ
写入192.20.57.188:1min--day_line-受影响行数为3473行!
写入192.20.57.

: 

In [11]:
df.query("date == '2024-02-26'")

,ID,code,date,tick,open,high,low,close,turnover,total_amount,update_time
trade_time,,,,,,,,,,,


In [ ]:
idx = list(range(0, len(df) // 2)) * 2
idx.sort()

In [ ]:
len(idx)

139440